![cnn](https://miro.medium.com/max/2510/1*vkQ0hXDaQv57sALXAJquxA.jpeg)

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [47]:
num_epochs = 10
batch_size = 4
learning_rate = 0.001

In [38]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [39]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                             transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [40]:
classes = ['plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

In [41]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # (3,32,32) -> (6,28,28) -> (6,14,14) -> (16,10,10)
        # -> (16,5,5) -> 16*5*5 -> 120 -> 84 -> 10
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, X):
        out = self.pool(self.conv1(X).relu())
        out = self.pool(self.conv2(out).relu())
        out = out.view(-1,16*5*5)
        out = self.fc1(out).relu()
        out = self.fc2(out).relu()
        out = self.fc3(out)
        return out

In [42]:
model = ConvNet()

In [43]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [48]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'epoch [{epoch+1}/{num_epochs}], Loss : {loss:.4f}')

epoch [1/10], Loss : 1.9059
epoch [2/10], Loss : 1.9079
epoch [3/10], Loss : 1.9119
epoch [4/10], Loss : 1.9552
epoch [5/10], Loss : 1.9423
epoch [6/10], Loss : 1.9323
epoch [7/10], Loss : 1.9048
epoch [8/10], Loss : 1.8609
epoch [9/10], Loss : 1.8144
epoch [10/10], Loss : 1.7530


In [52]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0]*10
    n_class_samples = [0]*10
    for images, labels in test_loader:
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    
    acc = 100. * n_correct / n_samples
    print(f'Accuracy of the network: {acc}%')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc}%')

Accuracy of the network: 62.630001068115234%
Accuracy of plane: 64.6%
Accuracy of car: 79.3%
Accuracy of bird: 53.6%
Accuracy of cat: 53.0%
Accuracy of deer: 58.5%
Accuracy of dog: 42.3%
Accuracy of frog: 61.4%
Accuracy of horse: 77.1%
Accuracy of ship: 71.7%
Accuracy of truck: 64.8%
